In [1]:
from tds import load_current_profiles

In [2]:
shear = 0.0027162e-12
pixel_size = 5.435999948531389e-6

In [4]:
path = "original/20210221-01_30_17_scan_phase1.pcl"

In [6]:
profiles = load_current_profiles(path, shear, pixel_size)

In [8]:
profiles[0]

array([[-1.87123921e+12, -1.86923789e+12, -1.86723656e+12, ...,
         1.86523524e+12,  1.86723656e+12,  1.86923789e+12],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])